In [ ]:
import re  
p1 = '[A-Z][a-z1-9]+([A-Z][a-z1-9]+)+'  # CLASS NAME
p2 = '[[][A-Z]{3}[-][0-9]{4}[]]'  # '[ASD-0522]' etc
p3 = '[a-z]+([-][a-z]+)+' # jekyll-relative-links etc

instance_name_list = ['FUNCTION','FILE','VARIABLE','CONSTANT']
words_list_1 = ['instead of', 'instead', 'so that', 'So that', 'not', 'For example', 'for example','that','old', 'when','before',
                'so','So', 'since', 'in order to', 'In order to', 'previous', 'support', 'like','also','Also','even after','as soon as possible',
                'unless','standardize','that','no','root','return','returns',"isn't", "wasn't",'convert','Converted','did not','was','non-root',
                'console.logs','can','now','by default','expires','which','they','during','converted']
words_list_2 = ['or', 'new', 'after', 'prepare for', 'if','examples','no longer','only','Only','once','but','as','even','good','open','while']
words_list_3 = ['Add','Adds','Added','build','Build','build:','simpler','improve','Improve', 'better', 'optimize','optim', 'initial']

for each_line in open(comment_file_path,'r', encoding="utf-8"):
    y = each_line.split() 

    # Case1: calcualte comment length
    comment_len = len(y) - 4
    y.append(' #CommentLength#')
    if comment_len <= 13:
        y.append('0')
    else:
        y.append('1')

    # Case2: if it is a merge format comment
    y.append(' #MergeComment#')
    if y[4] == 'Merge' or y[4] == 'Merge:':
        y.append('1')
    else:
        y.append('0')
            
    count_instance = 0
    count_fix = 0
    count_in = 0
    count_changes = 0
    exit_bug_num = 0
    exit_word_list_1 = False
    exit_word_list_2 = False
    exit_word_list_3 = False
    
    for idx, each_word in enumerate(y):        
        # Case3: number of instance of Function/File/Variable/Constant/Class name        
        v1 = re.match(p1,each_word,flags=0) #CLASS name handle
        v3 = re.match(p3,each_word,flags=0)
        if each_word in instance_name_list or v1 != None or v3 != None:
            count_instance += 1
                         
        # Case4: number of BUG_NUMBER : #234,(#32422), A[SV-2058]
        v2 = re.match(p2,each_word,flags=0)
        if each_word == 'BUG_NUMBER' or v2 != None:
            exit_bug_num += 1            
            
        if each_word.endswith('.') == True or each_word.endswith(',') == True or each_word.endswith(':') == True:
            each_word = each_word[:-1]            
        if each_word.startswith('(') == True or each_word.startswith('.') == True:
            each_word = each_word[1:]
        # handle phrases e.g.  prepare for, in order to
        if idx+2 < len(y):            
            two_consecutive_nums = each_word + ' ' + y[idx+1]
            three_consecutive_nums = each_word + ' ' + y[idx+1] + ' ' + y[idx+2]

            # Case5: if exit one of these words - words_list_1
            if each_word in words_list_1 or two_consecutive_nums in words_list_1 or three_consecutive_nums in words_list_1:
                exit_word_list_1 = True

            # Case6: if exit one of these words - words_list_2
            elif each_word in words_list_2 or two_consecutive_nums in words_list_2 or three_consecutive_nums in words_list_2:
                exit_word_list_2 = True
                
            # Case7: if exit one of these words - words_list_3
            elif each_word in words_list_3 or two_consecutive_nums in words_list_3 or three_consecutive_nums in words_list_3:
                exit_word_list_3 = True             

        # Case8: number of special words: fixed,in,changes..
        if each_word == 'fixed' or each_word == 'fix' or each_word == 'fixes':
            count_fix += 1
        elif each_word == 'in':
            count_in += 1
        elif each_word == 'changes':
            count_changes += 1
